In [2]:
#!pip3 install --upgrade pip
!pip3 install tifffile

Preamble, imports necessary libraries

In [3]:
import json
import os
import glob
import logging
from jsonschema import validate
import urllib.request
from pprint import pprint

In [4]:
#to be run if not installed yet
#!pip3 install pandas
!pip3 install pathlib
!pip3 install openpyxl
!pip3 install xlrd
!pip3 install xlwt
!pip3 install pyxlsb
!pip3 install hyperspy

In [5]:
# view all the metadata stored in the ZEISS TIFF file
# from https://github.com/ks00x/zeiss_tiff_meta.git
import zeiss_tiff_meta.zeisstiffmeta as zm

In [6]:
# Read in source image
src = 'FeMoOx_AntiA_04_1k5x_CN.tif'

# extract the full metadata as defined by Zeiss:
tab = zm.zeiss_meta(src)

# tab is a list of tuples, each tuple is a point of metadata

In [7]:
#print(tab)
#first element in the tuple is a name used by Zeiss, second one is the more human understandable form, 
#third is the value and the fouth is the unit, if it has a unit tab

In [8]:
del tab[0] # delete first result as it is useless

# 
resultDictionary = {**dict((x1+"_value",x2) for x0, x1, x2, x3 in tab) , **dict((x1+"_unit",x3) for x0, x1, x2, x3 in tab)}
resultDictionary_filtered={k: v for k, v in resultDictionary.items() if v != ''}
resultDictionary_filtered_sorted=dict(sorted(resultDictionary_filtered.items()))

In [87]:
resultDictionary = dict((x1, {"value":x2,"unit":x3}) for x0, x1, x2, x3 in tab)
# resultDictionary_filtered_sorted['Store resolution_value'].split('*')
resultDictionary_filtered_sorted

{'ASAP DFE1 Minus5V Analogue_unit': 'V',
 'ASAP DFE1 Minus5V Analogue_value': 0.0,
 'ASAP DFE1 Plus5V Analogue_unit': 'V',
 'ASAP DFE1 Plus5V Analogue_value': 0.0,
 'ASAP DFE1 Plus5V Digital_unit': 'V',
 'ASAP DFE1 Plus5V Digital_value': 0.0,
 'ASAP DFE1 Temperature_unit': '°C',
 'ASAP DFE1 Temperature_value': 0.0,
 'ASAP DFE2 Minus5V Analogue_unit': 'V',
 'ASAP DFE2 Minus5V Analogue_value': 0.0,
 'ASAP DFE2 Plus5V Analogue_unit': 'V',
 'ASAP DFE2 Plus5V Analogue_value': 0.0,
 'ASAP DFE2 Plus5V Digital_unit': 'V',
 'ASAP DFE2 Plus5V Digital_value': 0.0,
 'ASAP DFE2 Temperature_unit': '°C',
 'ASAP DFE2 Temperature_value': 0.0,
 'ASAP DFE3 Minus5V Analogue_unit': 'V',
 'ASAP DFE3 Minus5V Analogue_value': 0.0,
 'ASAP DFE3 Plus5V Analogue_unit': 'V',
 'ASAP DFE3 Plus5V Analogue_value': 0.0,
 'ASAP DFE3 Plus5V Digital_unit': 'V',
 'ASAP DFE3 Plus5V Digital_value': 0.0,
 'ASAP DFE3 Temperature_unit': '°C',
 'ASAP DFE3 Temperature_value': 0.0,
 'ASAP DFE4 Minus5V Analogue_unit': 'V',
 'ASAP D

In [10]:
#testing Nicolas' metadataschemas reader
from metadataSchemaReader import MetadataSchemaReader

In [11]:
mySchema = "SEM_schema.json"

with open(mySchema, 'r') as f:
    jsonSchema= json.load(f)
    
#print(type(jsonSchema["$defs"]),jsonSchema["$defs"])
readSchema=MetadataSchemaReader(jsonSchema)
#something wrong in conversion of the SEM json schema into a dictionary
#Q: how does the MetadataSchemaReader handle units?

In [12]:
schema = readSchema.searchedSchema

# list(schema["entry"].keys())

In [15]:
mySchema = "SEM_schema.json"

with open(mySchema, 'r') as f:
    jsonSchema= json.load(f)
    
#print(type(jsonSchema["$defs"]),jsonSchema["$defs"])
readSchema=MetadataSchemaReader(jsonSchema)
#something wrong in conversion of the SEM json schema into a dictionary
#Q: how does the MetadataSchemaReader handle units?

schema = readSchema.searchedSchema

# list(schema["entry"].keys())

print(schema["entry"]["instrument"]['eBeamSource']['accelerationVoltage']['value'])


float


In [16]:
import copy

# This function should take a path as a list of keys which points to the 
# inner key of the value which needs to be changed, and updates that value with val
# and returns nothing

# we have a nested dictionary called `schema` which needs to be walked and edited
def modVal(dic, keys, val):
    for key in keys[:-1]:
        dic = dic.setdefault(key, {})
    dic[keys[-1]] = val
    return None

# Key value pairs to test
key = 'entry.instrument.eBeamSource.accelerationVoltage.value'

print(schema["entry"]["instrument"]['eBeamSource']['accelerationVoltage']['value'])

op_dic = copy.deepcopy(schema)

key_path = key.split('.')

modVal(op_dic, key_path, 'V')

pprint(op_dic)

float
{'entry': {'definition': 'str',
           'endTime': 'str',
           'entryID': {'identifierType': 'str', 'identifierValue': 'str'},
           'instrument': {'FIB': {'FIBSpotSize': {'notes': 'str',
                                                  'qualifier': 'str',
                                                  'uncertainty': {'uncertaintyType': 'str',
                                                                  'value': 'float'},
                                                  'unit': 'str',
                                                  'value': 'float'},
                                  'angleToEBeam': {'notes': 'str',
                                                   'qualifier': 'str',
                                                   'uncertainty': {'uncertaintyType': 'str',
                                                                   'value': 'float'},
                                                   'unit': 'str',
                            

In [17]:
test_key2 = 'entry.instrument.FIB.iBeamSource.accelerationVoltage.value'

print(schema["entry"]["instrument"]['FIB']['iBeamSource']['accelerationVoltage']['value'])

op_dic = copy.deepcopy(schema)

key_path = test_key2.split('.')

modVal(op_dic, key_path, 'IBEAM99')

print(op_dic["entry"]["instrument"]['FIB']['iBeamSource']['accelerationVoltage']['value'])

float
IBEAM99


In [18]:
pprint(op_dic)

{'entry': {'definition': 'str',
           'endTime': 'str',
           'entryID': {'identifierType': 'str', 'identifierValue': 'str'},
           'instrument': {'FIB': {'FIBSpotSize': {'notes': 'str',
                                                  'qualifier': 'str',
                                                  'uncertainty': {'uncertaintyType': 'str',
                                                                  'value': 'float'},
                                                  'unit': 'str',
                                                  'value': 'float'},
                                  'angleToEBeam': {'notes': 'str',
                                                   'qualifier': 'str',
                                                   'uncertainty': {'uncertaintyType': 'str',
                                                                   'value': 'float'},
                                                   'unit': 'str',
                                  

In [19]:
# TODO
# Implement try and except for if key doesn't exist
# Check if right var type

with open('mapSEM.json', 'r') as m:
    mapSEM = json.load(m)

keys = list()
for i in mapSEM['entry']:
    keys.append(mapSEM['entry'][i])
    
vals = ['val1', 'val2', 'val3', 'val4']

keys

['entry.instrument.eBeamSource.accelerationVoltage.value',
 'entry.instrument.eBeamSource.accelerationVoltage.unit',
 'entry.instrument.eBeamSource.beamCurrent.unit',
 'entry.instrument.eBeamSource.beamCurrent.value']

In [20]:
new_dict = copy.deepcopy(schema)
# pprint(new_dict)

for i, key in enumerate(keys):
    modVal(new_dict, key.split('.'), vals[i])
    
pprint(new_dict)

{'entry': {'definition': 'str',
           'endTime': 'str',
           'entryID': {'identifierType': 'str', 'identifierValue': 'str'},
           'instrument': {'FIB': {'FIBSpotSize': {'notes': 'str',
                                                  'qualifier': 'str',
                                                  'uncertainty': {'uncertaintyType': 'str',
                                                                  'value': 'float'},
                                                  'unit': 'str',
                                                  'value': 'float'},
                                  'angleToEBeam': {'notes': 'str',
                                                   'qualifier': 'str',
                                                   'uncertainty': {'uncertaintyType': 'str',
                                                                   'value': 'float'},
                                                   'unit': 'str',
                                  

* Test function over list of keys and values to assign
* List of keys and values comes from `resultDictionary_filtered_sorted`
* A mapping needs to be done though to convert the md name to the schema name

This function should take as an input the desired metadata schema value name, find the corresponding value in the `resultDictionary_filtered__sorted`, and return the key-value pair

It is tested below:

In [57]:
testMap = {'ciao': 'hello', 'bella':'beautiful', 'vero':'true'} # akin to the SEMmap.json file in format 'schema_name':'SEM_name'
results = {'ciao': 5, 'bella': 3.2, 'vero':'test_val'}     # akin to the results_dict from the SEM

def get_keys_from_value(d, val):
    return [k for k, v in d.items() if v == val][0]

# Output should be the SEM metadata values but with the Schema names
def SEMtoSchema(results_dict, map_file, lookUp_var):
    # "translate" schema name into results_dict name
    schema_var_name = lookUp_var # name which should be looked up in map_file
    SEM_data_name   = get_keys_from_value(map_file, lookUp_var)
    corr_value      = results_dict[SEM_data_name]
    return (schema_var_name, corr_value)

SEMtoSchema(results, testMap, 'beautiful')

# get_keys_from_value(testMap, 'beautiful')

('beautiful', 3.2)

In [58]:
# testing SEMtoSchema() with actual map

mapFile = mapSEM['entry']

SEMtoSchema(resultDictionary_filtered_sorted, mapFile, 'entry.instrument.eBeamSource.accelerationVoltage.value')

# mapSEM['entry']

('entry.instrument.eBeamSource.accelerationVoltage.value', 5.0)

In [138]:
from attributeMapping import AttributeMapping

# Loading new map JSON

with open('new_mapSEM.json', 'r') as m:
    newmapSEM = json.load(m)

keys = list()
for i in newmapSEM['mappedTerms']:
    keys.append(newmapSEM['mappedTerms'][i])
    
metadataDict = copy.deepcopy(resultDictionary_filtered_sorted)

Map = AttributeMapping(metadataDict, newmapSEM, 'mappedTerms')
# TODO
# Combine endTime.Date and endTime.Time into one line called entry.endTime in ISO 8601
# make another entry from entry.instrument.imaging.numberOfPixels.xPixels called -.yPixels and split as obvious as float
# entry.instrument.imaging..tiltCorrection -> if True, value should be empty string, else 'None'

import datetime as dt
from dateutil import parser


mappedDict = Map.__dict__

# Make endTime var as ISO 8601
endTime = mappedDict['entry.endTime.Date'] + ' ' + mappedDict['entry.endTime.Time']
mappedDict['entry.endTime'] = parser.parse(endTime).isoformat()
mappedDict.pop('entry.endTime.Date')
mappedDict.pop('entry.endTime.Time')
# End Date fixing

# Split pixel sizes into x and y

imgSize = [float(s.strip()) for s in mappedDict['entry.instrument.imaging.numberOfPixels.xPixels'].split('*')]
mappedDict['entry.instrument.imaging.numberOfPixels.xPixels'] = imgSize[0]
mappedDict['entry.instrument.imaging.numberOfPixels.yPixels'] = imgSize[1]


# Print alphabetically-sorted new dict
{key: value for key, value in sorted(mappedDict.items())}



#TODO first thing tomorrow: tiltCorrection, run it through the JSON-maker

{'entry.endTime': '2021-07-13T18:23:36',
 'entry.instrument.FIB.FIBSpotSize.unit': 'nm',
 'entry.instrument.FIB.FIBSpotSize.value': 93.89,
 'entry.instrument.FIB.angleToEBeam.unit': '°',
 'entry.instrument.FIB.angleToEBeam.value': 54.0,
 'entry.instrument.FIB.gasInjectionSystem.GISName': 'None',
 'entry.instrument.FIB.gunPressure.unit': 'mbar',
 'entry.instrument.FIB.gunPressure.value': 3.73e-08,
 'entry.instrument.FIB.iBeamSource.accelerationVoltage.unit': 'kV',
 'entry.instrument.FIB.iBeamSource.accelerationVoltage.value': 30.0,
 'entry.instrument.FIB.iBeamSource.beamCurrent.unit': 'µA',
 'entry.instrument.FIB.iBeamSource.beamCurrent.value': 2.0,
 'entry.instrument.FIB.iBeamSource.sourceLifetime.unit': 'µAh',
 'entry.instrument.FIB.iBeamSource.sourceLifetime.value': 2103.99,
 'entry.instrument.FIB.iBeamWorkingDistance_unit': 'mm',
 'entry.instrument.FIB.iBeamWorkingDistance_value': 0.0,
 'entry.instrument.chamberPressure.unit': 'mbar',
 'entry.instrument.chamberPressure.value': 2.16e

AttributeError: 'float' object has no attribute 'split'

 * we have a SEM image with some metadata
 * this metadata is in some illegible form stored inside the image according to the specs of the manufacturing company
 * we want to take out the metadata into some standardized schema that we have developed
 * which metadata values do we need
 
output is a json file which has the values
 
so we need some function that takes the metadata points we need, finds them in this nested dictionary, replaces their value with the value, and dumps them to a json file, 

function pseudocode

* loop through all metadata points
* check if value is needed, if needed, continue:
* find equivalent name (key) in json schema
* ensure var type matches expected
* replace value in nested dictionary with value from metadata points

* when loop done, write to json file
* somehow, we must check for missing metadata, and leave blank in output json file. take only those filled vaues and make this output file

We need to only use those elements which have values, when there is no value, don't write it into the json file

image has a lot of metadata dont need all of it, schema requires some stuff not included in the image which must be entered manually after uploading the output file. this is done by the interface

In [21]:
with open('example_output.json', 'w') as f:
    json.dump(schema, f)